In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [6]:
data = yf.download('AAPL', start='2020-01-01', end='2024-01-01')
data.reset_index(inplace=True) 
print(data.head())


[*********************100%***********************]  1 of 1 completed

Price        Date      Close       High        Low       Open     Volume
Ticker                  AAPL       AAPL       AAPL       AAPL       AAPL
0      2020-01-02  72.716080  72.776606  71.466820  71.721026  135480400
1      2020-01-03  72.009125  72.771752  71.783969  71.941336  146322800
2      2020-01-06  72.582893  72.621631  70.876060  71.127851  118387200
3      2020-01-07  72.241539  72.849216  72.021223  72.592586  108872000
4      2020-01-08  73.403633  73.706264  71.943744  71.943744  132079200


In [5]:
print("Hellow world ")

Hellow world 
